In [1]:
import pickle
import pandas as pd
import implicit
import lightfm
import scipy

import string
# Библиотека построения индекса приближенного поиска ближайших соседей
import annoy
import numpy as np

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split

C:\Users\Dmitry Trushin\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
with open('Product_dict.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
prod = pd.DataFrame.from_dict(data, orient='index', columns=['title'])

In [4]:
prod

,title
168308,(197312) Пакет-майка 25см х 45см 906
134832,(62448) Перекись водорода р-р наружн. 3% фл.по...
101384,(72183) Салициловая кислота р-р спирт 2% фл 40...
168570,(197309) Пакет 28см х 50см 906
146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906"
...,...
193603,(110044169) название -1
193484,(110050486) название -1
192459,(110020427) название -1
193510,(110047818) название -1


In [5]:
checks = pd.read_csv("чековые данные.csv")
checks.drop(['name'], axis=1, inplace=True)

C:\Users\Dmitry Trushin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
checks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000003 entries, 0 to 20000002
Data columns (total 8 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   sale_date_date           object 
 1   contact_id               object 
 2   shop_id                  float64
 3   product_id               float64
 4   product_sub_category_id  float64
 5   product_category_id      float64
 6   brand_id                 float64
 7   quantity                 object 
dtypes: float64(5), object(3)
memory usage: 1.2+ GB


In [7]:
checks['key_col'] = checks['sale_date_date'].apply(str) + "_" + checks['contact_id'].apply(str) + "_" + checks['shop_id'].apply(str)
checks.head()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00",2018-12-07_1260627_1455.0
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00",2018-12-07_198287_279.0
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00",2018-12-07_2418385_848.0
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00",2018-12-07_1285774_1511.0
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00",2018-12-07_1810323_1501.0


In [8]:
checks.dropna(inplace=True)
checks = checks[checks['sale_date_date'].str.startswith("20")]
checks.reset_index(inplace=True, drop=True)
checks['sale_date_date'] = pd.to_datetime(checks['sale_date_date'])
checks.sort_values('sale_date_date', inplace=True)

# Разделим подборку на трейн и тест
train, test = train_test_split(checks, test_size=0.2)

Однако по моему тут неприменимы технические тесты и валидация из-за того, что используется item-item рекоммендации
Можно разделить по времени покупки, но зачем?
Будут результативнее бизнес метрики

Предобработаем выборку

In [9]:
# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [10]:
sentences = [preprocess_txt(k) for k in data.keys()]

# Подготовим кандидатогенератор, который будет отдавать фильмы похожие по текстовому описанию на те, .
# которые оенил пользователь
# Обучим Fasttext и заэмбедим фильмы
modelFT = FastText(sentences=sentences, min_count=1, window=5)

# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_names = annoy.AnnoyIndex(20 ,'angular')

# Будем хранить соответствия не только id-> продукт, но и продукт-> id, чтобы быстрее находить эмбеддинги продуктов

index_map = {}
reverse_index_map = {}
counter = 0

for i in tqdm_notebook(data.keys()):
    n_ft = 0
    reverse_index_map[counter] = i
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in preprocess_txt('i'):
        if word in modelFT.wv.key_to_index:
            vector_ft += modelFT.wv.key_to_index[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_names.add_item(counter, vector_ft)
    counter += 1

# 
ft_index_names.build(10)

C:\Users\DMITRY~1\AppData\Local\Temp/ipykernel_14128/3268327387.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(data.keys()):


  0%|          | 0/30418 [00:00<?, ?it/s]

True

In [11]:
reverse_item_mapping = {v: k for k, v in data.items()}

def recommend(items_list):
    current_vector = np.zeros(20)
    c = len(items_list)
    for iid in items_list:
        iname = data[iid]
        if iname not in reverse_index_map:
            c -= 1
            continue
        current_vector += np.array(ft_index_names.get_item_vector[reverse_index_map[iname]])
    return [data[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]

In [12]:
recommend(['52539', '110342'])

C:\Users\DMITRY~1\AppData\Local\Temp/ipykernel_14128/2382415262.py:12: RuntimeWarning: invalid value encountered in true_divide
  return [data[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]


['(114236) Трусопт капли глазн 2% 5мл фл N1 575',
 '(42233) Сенаде тб 13.5мг N500 598',
 '(103503) Перца водяного экстракт жидкий фл. 25мл 613',
 '(13405) Энцефабол сусп оралн 80,5мг/5мл фл 200мл N1 638',
 '(90364) Тербинафин таб.250мг №14 622',
 '(44128) Панадол таб.п.п.о.500мг №12 696',
 '(68258) Проваг капс. №10 448',
 '(13695) Люголя раствор с глицерином р-р фл 25г N1 404',
 '(36531) Пропанорм таб.п/о 150мг N50 738',
 '(67119) Суматриптан таб.п.п.о.100мг №2 398']

# Генерация кандидатов

Сгенерируем кандидатов

In [13]:
# Сделаем кандидатогенератор по категориям препаратов
medicines = {}
categories_dict = {}
medicines_p={}
for i in range(len(checks)):
    categories = str(checks.loc[i, "product_category_id"]).strip()
    if categories not in medicines:
        medicines[categories] = []
    medicines[categories].append(checks.loc[i, 'product_id'])
    medicines_p[checks.loc[i, 'product_id']]=categories 

In [14]:
# Сделаем рандомный кандидатогенератор 
def sample_random(sample_set, num_sampled=10):
    return np.random.choice(sample_set, num_sampled, replace=False)

In [45]:
products_unique=checks['product_id'].unique()
def candidate_generator(product_id):
    for i in product_id:
            category=medicines_p.get(int(i))
            candidates_categories = list(sample_random(medicines[str(category)]))
    return candidates_categories

In [46]:
# Проверим как работает наш кандидатогенератор
candidate_generator(['99821', '138583', '45321'])

[118364.0,
 146109.0,
 120601.0,
 82685.0,
 81946.0,
 33993.0,
 154101.0,
 148629.0,
 146507.0,
 138536.0]

Воспользуемся регулярными выражниями чтобы взять из результата модели FastText только id

In [19]:
import re

In [20]:
out=list(recommend(['52539']))
pattern = r'(?<=\().+?(?=\))'
id_only=[]
for i in out:
    med_id = re.search(pattern, i)
    id_only.append(med_id[0])

C:\Users\DMITRY~1\AppData\Local\Temp/ipykernel_14128/2382415262.py:12: RuntimeWarning: invalid value encountered in true_divide
  return [data[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]


In [21]:
id_only

['114236',
 '42233',
 '103503',
 '13405',
 '90364',
 '44128',
 '68258',
 '13695',
 '36531',
 '67119']

# Word2Vec

Обучим Эмбеддинги

In [22]:
from gensim.models import Word2Vec

train['product_id'] = train['product_id'].apply(int).apply(str)

grouped = train.groupby('key_col')
sentences = []

Сформируем последовательности для W2V

In [23]:
from tqdm import tqdm
sentences = []
for group in tqdm_notebook(grouped.groups):
    products = grouped.get_group(group)['product_id'].values
    if len(products) < 4:
        continue
    sentences.append(list(products))

C:\Users\DMITRY~1\AppData\Local\Temp/ipykernel_14128/3499826910.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for group in tqdm_notebook(grouped.groups):


  0%|          | 0/5723906 [00:00<?, ?it/s]

Обучим модель

In [24]:
modelW2V = Word2Vec(sentences, vector_size=10)

In [25]:
def recommend_w2v(items_list):
    current_vector = np.zeros(10)
    c = len(items_list)
    for iid in items_list:
        iname = data[iid]
        if iname not in modelW2V.wv:
            c -= 1
            continue
        current_vector += np.array(modelW2V.wv[reverse_index_map[iname]])
    return [data[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]

Проверим метод

In [26]:
recommend_w2v(['99821', '138583', '45321'])

C:\Users\DMITRY~1\AppData\Local\Temp/ipykernel_14128/2409218459.py:10: RuntimeWarning: invalid value encountered in true_divide
  return [data[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]


['(89165) Неулептил р-р д/приема внутрь 4% фл. 30мл 637',
 '(73870) Протопик мазь для наружного прим. 0,03% туба 30гр. уп. №1 467',
 '(31725) Афлодерм мазь д/наружн.прим.0,05% туба 20г №1 467',
 '(70301) Аспера Эфирное масло Анис 10мл 410',
 '(111856) Эндокринол Йод капс.№30 458',
 '(270981) Ринза таб.№4 703',
 '(120222) Ретинорм капс.500мг №30 574',
 '(68790) Энап таб.10мг №20 738',
 '(35808) Зитролид 250мг капс N6 403',
 '(112651) Зубная паста "Лакалют Бэйсик" Цитрус туба 75мл N1 529']

Применим также функцию, оставляющую лишь айди

In [39]:
def only_id(out):
    pattern = r'(?<=\().+?(?=\))'
    id_only=[]
    for i in out:
        med_id = re.search(pattern, i)
        id_only.append(int(med_id[0]))
    return id_only

Оказалось, айди препарата - это не число в скобках, а индекс строки в data

In [122]:
def id_2(out):
    ids = []
    for i in out:
        ids.append(int(prod[prod.title == i].index[0]))
    return ids
                   

In [123]:
id_2(recommend_w2v(['99821', '138583', '45321']))

C:\Users\DMITRY~1\AppData\Local\Temp/ipykernel_14128/2409218459.py:10: RuntimeWarning: invalid value encountered in true_divide
  return [data[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]


[88819, 51595, 144781, 70690, 103002, 350340, 105917, 151844, 84063, 34238]

Объединим все списки перед ранжированием

In [124]:
Propose = id_2(recommend_w2v(['99821', '138583', '45321']))+id_2(recommend(['99821', '138583', '45321']))+candidate_generator(['99821', '138583', '45321'])

C:\Users\DMITRY~1\AppData\Local\Temp/ipykernel_14128/2409218459.py:10: RuntimeWarning: invalid value encountered in true_divide
  return [data[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]
C:\Users\DMITRY~1\AppData\Local\Temp/ipykernel_14128/2382415262.py:12: RuntimeWarning: invalid value encountered in true_divide
  return [data[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]


In [125]:
for proposition in Propose:
    proposition = int(proposition)

In [126]:
Propose

[88819,
 51595,
 144781,
 70690,
 103002,
 350340,
 105917,
 151844,
 84063,
 34238,
 76206,
 72448,
 102027,
 131412,
 83447,
 138089,
 41273,
 35317,
 103865,
 65857,
 99144.0,
 75208.0,
 154101.0,
 154101.0,
 94449.0,
 81946.0,
 81946.0,
 105098.0,
 146507.0,
 154101.0]

# Ранжирование

Метод 1

In [127]:
# У нас есть список кандидатов, сгенерированных разными способами. Сначала выведем те, которые повторяются несколько раз
import collections
Pr_dict=collections.Counter(Propose)
Pr_dict=dict(sorted(Pr_dict.items(), key=lambda item: item[1], reverse = True))

In [128]:
Pr_dict

{154101.0: 3,
 81946.0: 2,
 88819: 1,
 51595: 1,
 144781: 1,
 70690: 1,
 103002: 1,
 350340: 1,
 105917: 1,
 151844: 1,
 84063: 1,
 34238: 1,
 76206: 1,
 72448: 1,
 102027: 1,
 131412: 1,
 83447: 1,
 138089: 1,
 41273: 1,
 35317: 1,
 103865: 1,
 65857: 1,
 99144.0: 1,
 75208.0: 1,
 94449.0: 1,
 105098.0: 1,
 146507.0: 1}

In [167]:
#Выведем первые 5 значений
import itertools
answer = dict(itertools.islice(Pr_dict.items(), 5)).keys()

In [160]:
print(answer)

dict_keys([154101.0, 81946.0, 88819, 51595, 144781])


Главный минус состоит в том, что методы могут давать неповторяющиеся варианты ответа, и тогда мы получим все ту же случайную выдачу

Метод 2

In [129]:
 #Попробуем ранжировать по популярности лекарства

In [130]:
checks_upd = checks[['product_id','quantity']]
checks_upd["quantity"] = checks_upd["quantity"].str.replace(',', '.', regex=False).astype(float)

C:\Users\DMITRY~1\AppData\Local\Temp/ipykernel_14128/1098190461.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checks_upd["quantity"] = checks_upd["quantity"].str.replace(',', '.', regex=False).astype(float)


In [131]:
checks_upd = checks_upd.groupby("product_id").sum().reset_index()

In [153]:
freq_dict={}
pop = []
for i in Propose:
    pop.append(checks_upd[checks_upd['product_id']==i].values[0][1])
Pr_dict_2 = dict(zip(Propose, pop))
Pr_dict_2=dict(sorted(Pr_dict_2.items(), key=lambda item: item[1], reverse = True))

In [154]:
Pr_dict_2

{81946.0: 44864.0,
 105098.0: 12230.343,
 154101.0: 11934.0,
 146507.0: 8704.0,
 103865: 8641.0,
 99144.0: 8558.0,
 75208.0: 5670.0,
 76206: 4555.0,
 65857: 2366.0,
 72448: 2293.834,
 35317: 1871.0,
 138089: 1779.0,
 94449.0: 1438.0,
 83447: 1074.0,
 41273: 887.0,
 102027: 678.0,
 131412: 360.0,
 88819: 92.0,
 103002: 77.0,
 70690: 70.0,
 105917: 69.0,
 350340: 67.0,
 144781: 66.0,
 51595: 61.0,
 151844: 59.0,
 84063: 58.0,
 34238: 39.0}

In [161]:
#Выведем первые 5 значений
answer_2 = dict(itertools.islice(Pr_dict_2.items(), 5)).keys()

In [162]:
print(answer_2)

dict_keys([81946.0, 105098.0, 154101.0, 146507.0, 103865])


# Второй этап ранжирования

Сначала будем выводить значения, которые были выведены в обоих методах

In [164]:
import collections
c=collections.Counter(list(answer_2)+list(answer))
Final_dict=dict(sorted(c.items(), key=lambda item: item[1], reverse = True))

In [165]:
Final_dict

{81946.0: 2,
 154101.0: 2,
 105098.0: 1,
 146507.0: 1,
 103865: 1,
 88819: 1,
 51595: 1,
 144781: 1}

Выведем первые 5 значений

In [168]:
Final = dict(itertools.islice(Pr_dict_2.items(), 5)).keys()
print(list(Final))

[81946.0, 105098.0, 154101.0, 146507.0, 103865]
